### add rsid mapping

In [99]:
import pandas as pd
import os
import numpy as np

In [100]:
ancestry=pd.read_csv("../data/supplemental/tables3.csv")
metal=pd.read_csv("../data/supplemental/tables4.csv")

In [101]:
chrom=pd.read_csv("/cellar/users/mpagadal/data/dbsnp/grch38.chrom.mapping.txt",header=None)
chrom=chrom.loc[0:23]
chrom[1]=chrom[0].str.split(".").str[0].str.split("_").str[1].astype(int)
mp_chrom=dict(zip(chrom[1],chrom[0]))

In [102]:
df_snps=pd.DataFrame({"snps":metal["SNP"].tolist()+ancestry["SNP"].tolist()})
df_snps["snps"]=df_snps["snps"].str.replace("chr","")
df_snps["chr"]=df_snps["snps"].str.split(":").str[0]
df_snps["chr"]=df_snps["chr"].replace("X",23)
df_snps["chr"]=df_snps["chr"].astype(int)
df_snps["bp1"]=df_snps["snps"].str.split(":").str[1]
df_snps["bp2"]=df_snps["bp1"].astype(int)+1
df_snps["bp1"]=df_snps["bp1"].astype(int)-1
df_snps["chr_id"]=df_snps["chr"].map(mp_chrom)
df_snps[["chr_id","bp1","bp2"]].to_csv("../data/dbsnp/map.bed",header=None,index=None,sep="\t")

In [103]:
chrom=pd.read_csv("/cellar/users/mpagadal/data/dbsnp/grch38.chrom.mapping.txt",header=None)
chrom=chrom.loc[0:23]
chrom[1]=chrom[0].str.split(".").str[0].str.split("_").str[1].astype(int)
mp_chrom=dict(zip(chrom[0],chrom[1]))

In [104]:
rsid=pd.read_csv("../data/dbsnp/rsid.txt",comment="#",delimiter="\t",header=None)
rsid["chr"]=rsid[0].map(mp_chrom)
rsid["chr"]=rsid["chr"].replace(23,"X")
rsid["ID"]=rsid["chr"].astype(str)+":"+rsid[1].astype(str)
mp_rsid=dict(zip(rsid["ID"],rsid[2]))

In [105]:
metal["ID"]=metal["SNP"].str.rsplit(":",2).str[0]
metal["ID"]=metal["ID"].str.replace("chr","")
metal["rsid"]=metal["ID"].map(mp_rsid)

In [106]:
ancestry["ID"]=ancestry["SNP"].str.rsplit(":",2).str[0]
ancestry["ID"]=ancestry["ID"].str.replace("chr","")
ancestry["rsid"]=ancestry["ID"].map(mp_rsid)

### Get UKBB analyses

In [107]:
ancestry_ukbb=pd.read_csv("../data/supplemental/tables3.ukbb.csv")

In [108]:
ancestry_ukbb=ancestry_ukbb[["MVP_id","hg19_coord","SNP","beta","standard_error","p_value","pheno"]]
ancestry_ukbb["MVP_id"]="chr"+ancestry_ukbb["MVP_id"]
ancestry_ukbb.columns=["SNP","hg19_coord","ukbb_proxy","ukbb_beta","ukbb_se","ukbb_p","pheno"]

In [109]:
ancestry_ukbb=ancestry_ukbb.drop_duplicates()

In [110]:
ancestry=pd.merge(ancestry,ancestry_ukbb,on=["SNP","pheno"],how="left")

In [112]:
del ancestry["Unnamed: 0"]

In [113]:
ancestry.to_csv("../data/supplemental/tables3.final.csv",sep=";")

In [32]:
metal_ukbb=pd.read_csv("../data/supplemental/tables4.ukbb.csv")
metal_ukbb=metal_ukbb[["MVP_id","hg19_coord","SNP","beta","standard_error","p_value","pheno"]]
metal_ukbb["MVP_id"]="chr"+metal_ukbb["MVP_id"]
metal_ukbb.columns=["SNP","hg19_coord","ukbb_proxy","ukbb_beta","ukbb_se","ukbb_p","pheno"]

In [34]:
metal=pd.merge(metal,metal_ukbb,on=["SNP","pheno"],how="left")

In [35]:
stats=pd.read_csv("../data/supplemental/tables4.group.stats.csv")

In [36]:
stats_final=pd.DataFrame()
for x in ["total","free","shbg","hypogonad"]:
    stats_filt=stats[stats["pheno"]==x]
    beta=pd.pivot(index="ID",columns="group",values="BETA",data=stats_filt)
    beta=beta.reset_index()
    beta["pheno"]=x
    se=pd.pivot(index="ID",columns="group",values="SE",data=stats_filt)
    se=se.reset_index()
    beta=pd.merge(beta,se,on="ID", suffixes=("_beta","_se"))
    a1=pd.pivot(index="ID",columns="group",values="A1",data=stats_filt)
    a1=a1.reset_index()
    beta=pd.merge(beta,a1,on="ID")
    stats_final=stats_final.append(beta)

In [37]:
stats_final=stats_final.rename(columns={"ID":"SNP"})

In [38]:
metal=pd.merge(metal,stats_final,on=["SNP","pheno"],how="left")

In [39]:
del metal["SP2"]

In [40]:
metal.to_csv("../data/supplemental/tables4.final.csv",index=None,sep=";")

### make MR input

In [21]:
direct="../data/summarystats/extract_/"
compiled_stats=pd.DataFrame()
for x in [x for x in os.listdir(direct)]:
    df=pd.read_csv(direct+x,delimiter="\t")
    df["file"]=x
    df["pheno"]=x.split(".")[2]
    df["group"]=x.split(".")[1]
    if x.split(".")[2]=="hypogonad":
        df["BETA"]=np.log(df["OR"])
        df["SE"]=df["LOG(OR)_SE"]
    compiled_stats=compiled_stats.append(df)
compiled_stats["SNP"]=compiled_stats["ID"]

In [41]:
for group in ["EUR","AFR","AMR","EAS"]:
    check=compiled_stats[compiled_stats["group"]==group]
    df=pd.merge(metal[["SNP","pheno","rsid"]],check,on=["SNP","pheno"],how="left")[["rsid","BETA","SE","A1","P","pheno"]]
    df=df[~df["BETA"].isnull()]
    for pheno in df["pheno"].unique():
        df[df["pheno"]==pheno][["rsid","BETA","SE","A1","P"]].to_csv(group+"."+pheno+".sumstats.forMR.csv",index=None)